In [1]:
import scipy.io as spio
import numpy as np

In [2]:
X = spio.loadmat(file_name='../data/bci/x.mat')

In [3]:
Y = spio.loadmat(file_name='../data/bci/y.mat')

In [4]:
X.keys()

dict_keys(['Intensification_Data', '__header__', '__globals__', '__version__'])

In [5]:
Y.keys()

dict_keys(['__header__', '__globals__', '__version__', 'Intensification_SType'])

In [6]:
X['Intensification_Data'].shape

(6480, 192, 64)

In [7]:
Y['Intensification_SType'].shape

(6480, 192)

In [8]:
X_train = X['Intensification_Data']
Y_train = Y['Intensification_SType'].mean(axis=1)

In [9]:
X_train.dtype, Y_train.dtype, X_train.shape, Y_train.shape

(dtype('<f8'), dtype('float64'), (6480, 192, 64), (6480,))

In [10]:
# HAR classification 
# Author: Burak Himmetoglu
# 8/15/2017

import pandas as pd

def standardize(train, test):
	""" Standardize data """

	# Standardize train and test
	X_train = (train - np.mean(train, axis=0)[None,:,:]) / np.std(train, axis=0)[None,:,:]
	X_test = (test - np.mean(test, axis=0)[None,:,:]) / np.std(test, axis=0)[None,:,:]

	return X_train, X_test

def one_hot(labels, n_class = 6):
	""" One-hot encoding """
	expansion = np.eye(n_class)
	y = expansion[:, labels-1].T
	assert y.shape[1] == n_class, "Wrong number of labels!"

	return y

def get_batches(X, y, batch_size = 100):
	""" Return a generator for batches """
	n_batches = len(X) // batch_size
	X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

	# Loop over batches and yield
	for b in range(0, len(X), batch_size):
		yield X[b:b+batch_size], y[b:b+batch_size]

In [11]:
np.mean(X_train, axis=0)[None,:,:].shape, np.mean(X_train, axis=0).shape, 
np.mean(X_train, axis=0)[None, None, :, :].shape, np.mean(X_train, axis=0).dtype

((1, 1, 192, 64), dtype('<f8'))

In [12]:
def normalize(X):
	""" Standardize data """

	# Standardize train and test
	X_norm = (X - np.mean(X, axis=0)[None,:,:]) / np.std(X, axis=0)[None,:,:]

	return X_norm

In [13]:
X_norm = normalize(X=X_train)

In [14]:
X_norm.shape, X_norm.dtype

((6480, 192, 64), dtype('<f8'))

In [15]:
# Deviding the input data into train and validation
length = X_norm.shape[0]//10
X_train_norm = X_norm[: -length]
X_valid_norm = X_norm[-length:]

In [16]:
X_train_norm.shape, X_valid_norm.shape, X_train_norm.dtype, X_valid_norm.dtype

((5832, 192, 64), (648, 192, 64), dtype('<f8'), dtype('<f8'))

In [17]:
Y_train_norm = Y_train[ :-length]
Y_valid_norm = Y_train[-length: ]

In [18]:
Y_train_norm.shape, Y_train_norm.dtype, Y_valid_norm.shape, Y_valid_norm.dtype

((5832,), dtype('float64'), (648,), dtype('float64'))

In [23]:
Y_valid_norm = np.array(Y_valid_norm, dtype=int)
Y_train_norm = np.array(Y_train_norm, dtype=int)

In [27]:
Y_valid_norm.dtype, Y_train_norm.dtype

(dtype('int64'), dtype('int64'))

In [28]:
Y_train_onehot = one_hot(labels=Y_train_norm, n_class=2)